In [1]:
!pip install fasttext
!pip install levenshtein

import fasttext
import fasttext.util
from Levenshtein import distance

import urllib.request
import json

In [2]:
# Load model
# fasttext.util.download_model('fr', if_exists='ignore')
ft = fasttext.load_model('cc.fr.300.bin')

# Load words
words = []
with open('/content/freq_word_clean.txt', 'r') as file:
    words = file.read().splitlines()

In [4]:
# Look how the model behave on just a few words
for word in words[:5]:
    neighbors = ft.get_nearest_neighbors(word)
    for d, n in neighbors:
        n = n.split('.')[0]
    print(word, [n for _, n in neighbors])

bleu ['violet', 'jaune', 'rouge', 'bleu.Le', 'gris', 'vert', 'blanc', 'noir', 'turquoise', 'mauve']
super ['sympa', 'suuuuuper', 'suuuuuuper', 'très', 'suuuuper', 'hyper', 'génial', 'suuuper', 'chouette', 'supers']
autre ['part', 'quelque', 'chose', 'une', 'autre.Une', 'quelconque', 'seule', "autre.D'", 'côté', 'même']
bizarre ['bizzare', 'étrange', 'bizare', 'bizard', 'marrant', 'bizarrement', 'bizzarre', 'chelou', 'bizzard', 'bizarre.']
difficile ['compliqué', 'difficle', 'facile', 'compliquée', 'impossible', 'ardu', 'difficiles', 'évident', 'ardue', 'diffcile']


Grâce au modèle de fasttext, on obtient une liste des 10 mots les plus rapprochés ainsi que leur distance associées. Dans notre cas, ces distances ne sont pas vraiment représentative d'une bonne association permettant une partie excitante d'Undercover. Il va falloir trouver un moyen de filtrer et réordonner ces mots dans un ordre qui nous convient mieux.

On peut également remarquer que parmis les mots voisins se trouvent le même mot ortographié de manières différentes (majuscules, ponctuation, pluriel). Commençons par écarter ses voisins ci. Pour filtrer, nous allons d'abord nous débarasser de la ponctuation (appelé stopwords) et comparer la distance de Levenshtein entre le mot d'origine et un voisin.

Pour illustrer le propos nous allons nous concentrer sur le mot: 'impossible'

In [6]:
example = 'impossible'
ft_neighbors = ft.get_nearest_neighbors(example)
example, ft_neighbors

('impossible',
 [(0.7440884113311768, 'difficile'),
  (0.7319589257240295, 'quasi-impossible'),
  (0.7296919822692871, 'impossibles'),
  (0.7182818651199341, 'impensable'),
  (0.6970745921134949, 'inconcevable'),
  (0.6799744367599487, 'impossible.'),
  (0.6762660145759583, 'impossible.Il'),
  (0.6699529886245728, 'possible'),
  (0.6698157787322998, 'inenvisageable'),
  (0.6677538156509399, 'envisageable')])

In [7]:
stopwords = '?,.;/:-_<>'

neighbors = {}
for d, n in ft_neighbors:
    # new_n = n
    for sw in stopwords:
        n = n.split(sw)[0]
    if distance(example, n) > 1 and n != '':
        neighbors[n] = {'distance':d}

neighbors

{'difficile': {'distance': 0.7440884113311768},
 'quasi': {'distance': 0.7319589257240295},
 'impensable': {'distance': 0.7182818651199341},
 'inconcevable': {'distance': 0.6970745921134949},
 'possible': {'distance': 0.6699529886245728},
 'inenvisageable': {'distance': 0.6698157787322998},
 'envisageable': {'distance': 0.6677538156509399}}

Maintenant que nous avons écarté les mots similaires. Il nous faut faire un choix entre ceux restant. Pour s'assurer de la cohérence de l'association des mots, nous partons sur le concept d'[hyponymie](https://fr.wikipedia.org/wiki/Hyponymie).

Nous allons garder, parmis les voisins restants, ceux qui possèdent un hypéronyme commun. Pour ce faire nous allons consulter la base de données [JeuxDeMots](https://www.jeuxdemots.org/jdm-about.php) pour regarder les relations d'hyponyme entre le mot d'origine et un voisin, si tout deux ont un mot similaire dans ces relations nous conserverons

In [8]:
URI = 'https://jdm-api.demo.lirmm.fr/v0'

def get_relations_type(word, relation):
    relations = []
    try:
        url = f'{URI}/relations/from/{urllib.parse.quote(word)}?types_ids={relation}'
        response = urllib.request.urlopen(url)
        content = json.load(response)
        for node in content['nodes']:
            relations.append(node['name'])
    except Exception as e:
        print(f'Bad request node={word}({relation}): "{e}"')
    return relations

def get_relations(word):
    relations = []
    relations.extend(get_relations_type(word, 7))
    relations.extend(get_relations_type(word, 8))
    relations.extend(get_relations_type(word, 10))
    relations.extend(get_relations_type(word, 35))
    relations.extend(get_relations_type(word, 36))
    return relations

In [9]:
hypo_word = get_relations(example)

for n in neighbors:
    hypo_n = get_relations(n)
    neighbors[n]['hypo'] = len(list(set(hypo_word) & set(hypo_n)))

neighbors

{'difficile': {'distance': 0.7440884113311768, 'hypo': 18},
 'quasi': {'distance': 0.7319589257240295, 'hypo': 11},
 'impensable': {'distance': 0.7182818651199341, 'hypo': 15},
 'inconcevable': {'distance': 0.6970745921134949, 'hypo': 19},
 'possible': {'distance': 0.6699529886245728, 'hypo': 16},
 'inenvisageable': {'distance': 0.6698157787322998, 'hypo': 18},
 'envisageable': {'distance': 0.6677538156509399, 'hypo': 16}}

On classe les voisins par la distance et le nombre de relation en communs, on sélectionne les 3 premiers pour former 3 associations de mots.

In [10]:
sorted_neighbors = dict(sorted(neighbors.items(), key=lambda item: item[1]['hypo'])[-3:])
with open('/content/undercover.txt', 'w+') as file:
    for neighbor in sorted_neighbors:
        file.write(f'{example}:{neighbor}\n')

On automatise le tout

In [18]:
def get_neighbors(word):
    stopwords = '?,.;/:-_<>'

    neighbors = {}
    for d, n in ft.get_nearest_neighbors(word):
        # new_n = n
        for sw in stopwords:
            n = n.split(sw)[0]
        if distance(word, n) > 1 and n != '':
            neighbors[n] = {'distance':d}
            hypo_n = get_relations(n)
            neighbors[n]['hypo'] = len(list(set(hypo_word) & set(hypo_n)))
    return neighbors

def get_undercover_relations(word):
    relations = []
    neighbors = get_neighbors(word)
    neighbors = dict(sorted(neighbors.items(), key=lambda item: item[1]['hypo'])[-3:])
    for n in neighbors:
        relations.append((word, n))
    return relations

In [19]:
undercover_file = open('/content/undercover.txt', 'w')
with open('/content/freq_word_clean.txt', 'r') as file:
    for word in file.read().splitlines():
        print(word)
        for w1, w2 in get_undercover_relations(word):
            print('\t', w1, w2)
            undercover_file.write(f'{w1}:{w2}\n')

bleu
	 bleu noir
	 bleu turquoise
	 bleu rouge
super
Bad request node=suuuuuper(7): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuper(8): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuper(10): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuper(35): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuper(36): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuuper(7): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuuper(8): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuuper(10): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuuper(35): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuuuper(36): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuper(7): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuper(8): "HTTP Error 500: Internal Server Error"
Bad request node=suuuuper(10): "HTTP Error 500: Internal Server Error"
Bad request nod